In [ ]:
# !pip install psycopg2-binary

In [31]:
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import os

In [2]:
DB_PARAMS = {
    "user": "postgres",
    "password": "123456",
    "host": "127.0.0.1",
    "port": "5432",
    "database": "postgres"  
}

DB_NAME = "lego_database"

In [9]:
def database_exists(conn, db_name):
    """
    Verifica si una base de datos ya existe.
    """
    with conn.cursor() as cur:
        cur.execute("SELECT 1 FROM pg_database WHERE datname=%s", (db_name,))
        return cur.fetchone() is not None

In [3]:
def execute_sql_from_file(conn, file_path, **kwargs):
    """
    Executes SQL commands from a given file.
    """
    with open(file_path, 'r') as file:
        sql = file.read().format(**kwargs)
    with conn.cursor() as cur:
        cur.execute(sql)

In [33]:
def create_database(connection_params, db_name):
    try:
        # Conectar a la base de datos predeterminada para realizar la verificación y creación
        conn = psycopg2.connect(**connection_params)
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        with conn.cursor() as cur:
            # Verificar si la base de datos ya existe
            cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", (db_name,))
            exists = cur.fetchone()
            if exists:
                print(f"Database '{db_name}' already exists.")
            else:
                # Ejecutar el comando CREATE DATABASE si no existe
                cur.execute(f"CREATE DATABASE {db_name}")
                DB_PARAMS["database"] = db_name
                print(f"Database '{db_name}' created successfully.")
        conn.close()

    except Exception as e:
        print(f"Error creating database: {e}")

In [42]:
def create_tables(db_params):
    """
    Creates tables in the database.
    """
    
    sql_files = [
    'create_colors_table.sql',
    'create_part_categories_table.sql',
    'create_parts_table.sql',
    'create_themes_table.sql',
    'create_sets_table.sql',
    'create_inventories_table.sql',
    'create_inventory_parts_table.sql',
    'create_inventory_sets_table.sql'
]

    try:
        with psycopg2.connect(**db_params) as conn:
            conn.set_session(autocommit=True)
            
            for sql_file in sql_files:
                execute_sql_from_file(conn, f'sql/{sql_file}')
                print(f"Table from '{sql_file}' created.")
    except Exception as e:
        print(f"Error creating tables: {e}")

In [43]:
create_database(DB_PARAMS, DB_NAME)

Database 'lego_database' already exists.


In [44]:
create_tables(DB_PARAMS)

Table from 'create_colors_table.sql' created.
Table from 'create_part_categories_table.sql' created.
Table from 'create_parts_table.sql' created.
Table from 'create_themes_table.sql' created.
Table from 'create_sets_table.sql' created.
Table from 'create_inventories_table.sql' created.
Table from 'create_inventory_parts_table.sql' created.
Table from 'create_inventory_sets_table.sql' created.


# Consultas.

In [36]:
def execute_query(db_params, query):
    """
    Ejecuta una consulta SQL en la base de datos especificada.

    Parámetros:
    - db_params: Un diccionario con los parámetros de conexión a la base de datos.
                 Ejemplo: {'dbname': 'mi_base_de_datos', 'user': 'usuario', 'password': 'contraseña', 'host': 'localhost'}
    - query: La consulta SQL a ejecutar como una cadena de texto.
    """
    try:
        # Establecer conexión con la base de datos
        conn = psycopg2.connect(**db_params)
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cursor = conn.cursor()
        
        # Ejecutar la consulta
        cursor.execute(sql.SQL(query))
        
        # Si la consulta es de selección, fetch results
        if query.strip().lower().startswith("select"):
            records = cursor.fetchall()
            return records
        else:
            print("Consulta ejecutada exitosamente.")
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error al ejecutar la consulta: {error}")
    finally:
        if conn is not None:
            cursor.close()
            conn.close()
            print("Conexión cerrada.")

In [37]:
query = "select * from colors"
execute_query(DB_PARAMS, query)

Conexión cerrada.


[]